### Question search engine

Помните первую ЛР, где вы использовали эмбеды GloVe для поиска вопросов (а-ля Retrieval на минималках)? Это было... мило, но далеко до SOTA. Займемся теперь по-серьезному с моделями, которые учитывают контекст?


In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
   

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/70.8M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/7.83M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

Мы случайно выбрали модель, обученную на QQP, но хороша ли она?

Одним из способов измерения является точность валидации - именно это вы и будете реализовывать дальше.

немного подсказок:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

Ваша задача - измерить точность валидации вашей модели.
Наивное измерение может занять несколько часов.

Пожалуйста, убедитесь, что вы используете следующие оптимизации:
- запуск модели на GPU с no_grad
- использовать batch size > 1
- используйте оптимизацию data loader при num_workers > 1
- (опционально) использовать [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)



In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import default_data_collator
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

batch_size = 64
num_workers = 8
val_set = qqp_preprocessed['validation']
val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=num_workers
)



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
accurate_predictions = 0
sample_count = 0

amp_context = torch.cuda.amp.autocast if torch.cuda.is_available() else lambda x: x

model.eval()
with torch.no_grad():
    for data_batch in tqdm(val_loader, desc="Validation Progress", leave=False):
        ids = data_batch['input_ids'].to(device)
        mask = data_batch['attention_mask'].to(device)
        type_ids = data_batch['token_type_ids'].to(device)
        true_labels = data_batch['labels'].to(device)

        with amp_context():
            model_outputs = model(
                input_ids=ids,
                attention_mask=mask,
                token_type_ids=type_ids
            )

        predictions = torch.argmax(model_outputs.logits, dim=1)

        accurate_predictions += (predictions == true_labels).sum().item()
        sample_count += true_labels.size(0)

Validation Progress:   0%|          | 0/632 [00:00<?, ?it/s]<ipython-input-9-af4ffdfc0919>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp_context():


In [ ]:
accuracy = accurate_predictions / sample_count
print(f"Validation Accuracy: {accuracy:.4f}")

assert 0.9 < accuracy < 0.91

Validation Accuracy: 0.9084


### Задание 2: обучение модели (4 балла)

Для выполнения этого задания у вас есть два варианта:

__Вариант A:__ жестко зафайтюнить модель [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Еще лучше выбрать лучшую модель, основанную на публичных бенчмарках (например, [GLUE](https://gluebenchmark.com/)).

Вы можете написать обучающий код вручную или использовать transformers.Trainer (см. [этот пример](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Пожалуйста, убедитесь, что точность вашей модели по крайней мере __сопоставима__ с приведенным выше примером для BERT.


__Вариант B:__ сравните не менее 3 зайфатюненных предобученных моедлей (в дополнение к приведенной выше модели BERT). Для каждой модели укажите (1) ее точность, (2) ее скорость, измеряемую в выборках в секунду на вашем оборудовании, и (3) ее размер в мегабайтах. Сравнивайте только в одинаковых условиях, на одинаковых CPU / GPU. Сведите результаты в таблицу и напишите короткий (~полстраницы поверх таблицы) фидбек о том, что получилось / почему не получилось


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

pretrained_model = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
classifier_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = qqp_preprocessed
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

train_args = TrainingArguments(
    output_dir="./model_output",
    eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="no",
    logging_dir="./training_logs",
    logging_steps=50,
    report_to="none",
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
)

trainer = Trainer(
    model=classifier_model,
    args=train_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-12-4bf0b1eea3b0>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2024-12-17 07:22:30,141] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss
50,0.658300
100,0.656600
150,0.651700
200,0.658000
250,0.664500
300,0.670700
350,0.661500
400,0.667300
450,0.661900
500,0.660400


TrainOutput(global_step=5686, training_loss=0.6390531478194842, metrics={'train_runtime': 3316.057, 'train_samples_per_second': 109.722, 'train_steps_per_second': 1.715, 'total_flos': 2.393340547233485e+16, 'train_loss': 0.6390531478194842, 'epoch': 1.0})

In [ ]:
classifier_model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/spm.model',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

### Task 3: try the full pipeline (1 point)

Наконец, пришло время использовать вашу модель для поиска дубликатов вопросов.

Пожалуйста, реализуйте функцию, которая берет вопрос и находит топ-5 потенциальных дубликатов в обучающем наборе.

Пока что не страшно, если ваша функция будет работать медленно, главное, чтобы она давала правильные результаты.

Покажите, как работает ваша функция, хотя бы на 5 примерах.


__Бонус (2 балла):__ для получения бонусных баллов попробуйте ускорить пайплайн.

 Например, вы можете сделать воронку. Сначала прогоняете тупую модель (логрег или что-нибудь еще), которая определяет потенциальных кандидатов на отбор, а затем уже свою махину использовать.

 Если вы хотите получить дополнительных баллов, пожалуйста, сохраните как оригинальную реализацию, так и оптимизированную - и кратко объясните, в чем разница. и ВО СКОЛЬКО РАЗ вам удалось ускорить
